## DEPLOYING MODEL

In [4]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

# Load trained pipeline
pipeline = joblib.load("pipeline.pkl")

# Set threshold
THRESHOLD = 0.4

# Create Flask app
app = Flask(__name__)

@app.route("/")
def home():
    return "✅ Twitter Sentiment Analysis API is running!"

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Expect JSON input: {"tweets": ["tweet1", "tweet2"]}
        data = request.get_json()

        if not data or "tweets" not in data:
            return jsonify({"error": "Invalid input. Send JSON: {'tweets': ['...']}"}), 400

        texts = data["tweets"]

        # Get prediction probabilities
        probs = pipeline.predict_proba(texts)[:, 1]

        # Apply threshold
        preds = (probs >= THRESHOLD).astype(int)

        # Map to labels
        labels = ["negative" if p == 0 else "positive" for p in preds]

        return jsonify({"predictions": labels, "probabilities": probs.tolist()})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [3]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

# Load your pre-trained pipeline
pipeline = joblib.load('pipeline.pkl')

# Threshold for negative class
THRESHOLD = 0.4

# Flask app
app = Flask(__name__)

# Home route
@app.route('/')
def home():
    return "Twitter Sentiment Analysis API is running!"

# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        texts = data['tweets']  # expects {"tweets": ["tweet1", "tweet2", ...]}
        
        # Get probabilities for positive class
        probs = pipeline.predict_proba(texts)[:, 1]
        
        # Apply threshold
        preds = (probs >= THRESHOLD).astype(int)
        labels = ["negative" if p == 0 else "positive" for p in preds]
        
        return jsonify({"predictions": labels})
    
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

Install Flask

If you haven’t yet:

pip install flask

2️⃣ Create the app structure
sentiment_app/
│
├─ app.py           # Main Flask app
├─ text_pipeline.pkl # Your pipeline (TF-IDF + model)
└─ requirements.txt # For deployment

3️⃣ Flask app template (app.py)
from flask import Flask, request, jsonify
import joblib

# Load your pre-trained pipeline (replace with final model later)
pipeline = joblib.load('text_pipeline.pkl')

# Optional: define threshold for negative class
THRESHOLD = 0.4

# Flask app
app = Flask(_name_)

# Home route
@app.route('/')
def home():
    return "Twitter Sentiment Analysis API is running!"

# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        texts = data['tweets']  # expects {"tweets": ["tweet1", "tweet2", ...]}
        
        # Get probabilities for positive class
        probs = pipeline.predict_proba(texts)[:,1]
        
        # Adjust threshold for class 0 if needed
        preds = (probs >= THRESHOLD).astype(int)
        labels = ["negative" if p==0 else "positive" for p in preds]
        
        return jsonify({"predictions": labels})
    
    except Exception as e:
        return jsonify({"error": str(e)})

# Run the app
if _name_ == '_main_':
    app.run(debug=True)

4️⃣ Test locally

Run the app:

python app.py


Send a POST request (you can use Postman or Python requests):

import requests

url = "http://127.0.0.1:5000/predict"
data = {"tweets": ["I love the new Apple iPhone!", "Android keeps crashing."]}

response = requests.post(url, json=data)
print(response.json())


Expected output:

{"predictions": ["positive", "negative"]}

5️⃣ Notes

You can replace text_pipeline.pkl with your final tuned model anytime — no changes to API code.

Threshold is easily adjustable (THRESHOLD = 0.4) to improve negative tweet detection.

This app is deployment-ready for Heroku, AWS, or any cloud provider

Install FastAPI + Uvicorn
pip install fastapi uvicorn

2️⃣ Create the app structure
sentiment_app/
│
├─ app.py           # FastAPI app
├─ text_pipeline.pkl # Your pipeline (TF-IDF + model)
└─ requirements.txt

3️⃣ FastAPI app template (app.py)
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np

# Load your pre-trained pipeline
pipeline = joblib.load('text_pipeline.pkl')

# Threshold for negative class
THRESHOLD = 0.4

# FastAPI app
app = FastAPI(title="Twitter Sentiment Analysis API")

# Define request model
class Tweets(BaseModel):
    tweets: list[str]

# Home route
@app.get("/")
def read_root():
    return {"message": "Twitter Sentiment Analysis API is running!"}

# Prediction route
@app.post("/predict")
def predict(data: Tweets):
    try:
        texts = data.tweets
        # Predict probabilities for positive class
        probs = pipeline.predict_proba(texts)[:,1]
        preds = (probs >= THRESHOLD).astype(int)
        labels = ["negative" if p==0 else "positive" for p in preds]
        return {"predictions": labels}
    
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

4️⃣ Run the app
uvicorn app:app --reload


--reload allows hot-reloading when you edit the code.

Open browser: http://127.0.0.1:8000 → you’ll see the home message.

Swagger docs: http://127.0.0.1:8000/docs → interactive API testing.

5️⃣ Test with Python
import requests

url = "http://127.0.0.1:8000/predict"
data = {"tweets": ["I love the new Apple iPhone!", "Android keeps crashing."]}
response = requests.post(url, json=data)
print(response.json())


Expected:

{"predictions": ["positive", "negative"]}

✅ Advantages of FastAPI

Auto-generates interactive docs (/docs) → great for demos.

Supports async calls → scalable for many requests.

Easy to swap in your final pipeline without changing any endpoint.

Works well with Docker for cloud deployment